# 한국어 LLM 증류 학습 실습


이 실습은 한국어 법률 QA 데이터셋(`jihye-moon/LawQA-Ko`)을 활용하여,
고성능 LLM(교사 모델)로부터 생성된 응답을 기반으로
경량 LLM(학생 모델)을 LoRA 방식으로 증류 학습하는 전 과정을 다룹니다.

이를 통해 다음을 실습합니다:
- 고품질 응답을 생성하는 교사 모델 설정 및 응답 필터링
- 학생 모델의 추론 전 성능 측정
- 증류 데이터 생성 및 Alpaca 포맷 변환
- LoRA 기반의 경량 파인튜닝 수행
- 학습 전후 학생 모델의 성능 비교 및 추론 테스트

🧠 LLM 증류란?


대규모 언어모델(LLM)은 뛰어난 성능을 보이지만,
계산 자원이 많이 들고, 실제 환경에서 사용하기엔 비효율적인 경우가 많습니다.


이를 해결하기 위해 LLM의 능력을 보다 작은 모델에 '전수'하는 과정을
Knowledge Distillation (지식 증류) 또는 LLM Distillation이라 부릅니다.


<br>

이번 실습은 전통적인 로짓 기반의 Knowledge Distillation과는 달리,  
**Teacher 모델의 출력 응답을 그대로 따라하도록 학습시키는 실용적인 증류 방식**입니다.

- Teacher: 고성능 LLM (예: LLaMA, text-davinci-003 등)
- Student: 경량 LLM (예: 3B/7B 모델)
- 학습 방식: Teacher의 **응답 텍스트를 target으로 하여** student가 이를 그대로 재현하도록 학습

이 방식은 **학습 구조가 간단하면서도 강력한 성능 개선**이 가능하여 실제 오픈소스 LLM 튜닝에 널리 활용됩니다.


참고논문

- https://arxiv.org/abs/2212.10560
- https://crfm.stanford.edu/2023/03/13/alpaca.html
- https://arxiv.org/abs/2210.14215

---

<br>

목차

1) 실습 준비
  - (1) 라이브러리 설치 및 환경 설정
  - (2) 데이터셋 로드 및 확인

<br>

2) 교사 모델 로드 및 증류 데이터 생성
  - (1) 교사 모델 로드 (float16)
  - (2) 응답 필터링 함수 정의
  - (3) 증류 데이터 생성 및 샘플 확인

<br>

3) 학생 모델 로드 및 학습 준비
  - (1) 학생 모델 로드 및 LoRA 설정
  - (2) 학습 데이터 구성 (Alpaca 포맷 적용)

<br>

4) 학생 모델 학습 및 저장
  - (1) 학습 설정 및 진행
  - (2) 모델 저장

<br>

5) 증류 결과 확인 및 비교
  - (1) 증류 전후 응답 비교
  - (2) 간단한 추론 테스트


  <br>

  ---

## 1) 실습 준비

이번 실습에서는 한국어 법률 QA 데이터셋을 활용하여 LLM을 증류 학습하는 과정을 진행합니다. 먼저, 필요한 라이브러리를 불러오고 사용할 데이터셋을 로드합니다.

<br>

### (1) 라이브러리 불러오기 및 환경 설정

이 실습에서는 LLM 증류 학습을 위해 Unsloth 라이브러리를 활용합니다.
Unsloth는 Hugging Face 기반의 모델을 더 빠르고 가볍게 학습할 수 있도록 최적화된 도구입니다.

먼저 !pip install unsloth 명령어를 통해 필요한 라이브러리를 설치한 후, 실습에 사용할 주요 라이브러리들을 불러옵니다.

설치 후 반드시 런타임 재시작이 필요합니다.

In [ ]:
!pip install unsloth # 실행 후 런타임 다시 시작

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import random
import numpy as np
from datasets import load_dataset
from tqdm import tqdm
import gc
from unsloth import FastLanguageModel
from transformers import TextStreamer

# GPU 확인
print(f"CUDA 사용 가능 여부: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"현재 GPU: {torch.cuda.get_device_name()}")
    print(f"가용 GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
CUDA 사용 가능 여부: True
현재 GPU: Tesla T4
가용 GPU 메모리: 14.74 GB


### (2) 데이터셋 로드 및 확인

이번 실습에서는 **한국어 법률 질문-응답 데이터셋인 jihye-moon/LawQA-Ko**를 사용합니다.
이 데이터는 질문(question)과 정답(answer) 형식으로 구성되어 있으며, 법률 상담 챗봇이나 QA 시스템 학습에 적합합니다.


<br>

데이터셋을 Hugging Face datasets 라이브러리를 사용해 로드하고, 전체 샘플 수와 샘플 내용을 확인합니다.



In [ ]:
# ✅ [1단계] 데이터 로드 및 확인
print("\n[1단계] 데이터 로드 중...")
dataset = load_dataset("jihye-moon/LawQA-Ko", split="train")  # 'train' split만 사용

# ✅ 전체 샘플 수 출력
print(f"데이터셋 크기: {len(dataset)}")

# ✅ 첫 번째 샘플 확인 (질문 + 정답 구조)
print("샘플:", dataset[0])


[1단계] 데이터 로드 중...
데이터셋 크기: 14819
샘플: {'question': '최저임금제도란 무엇이며, 최저임금액은 어떻게 결정되는지요?', 'precedent': '', 'answer': '최저임금제도란 국가가 임금액의 최저한도를 정하여 사용자에게 이를 준수하도록 강제하는 제도를 말합니다. 그러므로 최저임금액이 결정·고시되면 사용자는 근로자와 합의하여 최저임금액보다 낮은 임금을 지급하기로 약정하더라도 그것은 당연히 무효가 되며, 고용노동부장관이 고시한 최저임금액 이상을 지급하여야 합니다(최저임금법 제6조). 최저임금제도의 적용대상은 근로자를 사용하는 모든 사업 또는 사업장에 적용되며, 상용근로자 뿐만 아니라 임시근로자나 일용근로자, 시간제근로자 등 모든 근로자에게 적용됩니다. 다만, 동거의 친족만을 사용하는 사업과 가사사용인에 대하여는 적용하지 아니하고, 선원법의 적용을 받는 선원 및 선원을 사용하는 선박의 소유자에 대하여는 이를 적용하지 아니합니다(같은 법 제3조). 그리고 수습사용 중에 있는 자로서 수습사용한 날부터 3월 이내인 근로자는 시간급 최저임금액의 90%를 지급할 수 있고, 사용자가 고용노동부장관의 인가를 받은 감시 또는 단속적으로 근로에 종사하는 자(수위, 경비원, 자가용운전기사 등)는 시간급 최저임금액의 80%를 지급할 수 있습니다(같은 법 제5조 제2항, 같은 법 시행령 제3조).그러나 같은 법 시행령 제6조는 사용자는 고용노동부장관의 인가를 받아 ‘근로자의 정신 또는 신체의 장애가 당해 근로자를 종사시키고자 하는 업무의 수행에 직접적으로 현저한 지장을 주는 것이 명백하다고 인정되는 자’에 대하여는 최저임금의 적용을 제외할 수 있도록 규정하고 있습니다.고용노동부장관은 매년 3월 31일까지 근로자위원, 사용자위원, 공익위원 등으로 구성된 최저임금심의위원회에 최저임금에 관한 심의를 요청하여야 하고, 동 위원회에서는 근로자의 생계비, 유사근로자의 임금, 노동생산성 및 소득분배율 등을 고려하여 최저임금안을 심의하며, 심의위원회로부터 

## 2) 교사 모델 로드 및 증류 데이터 생성

교사 모델은 고성능 LLM으로, 정답 대신 사용할 고품질 응답을 생성해주는 역할을 합니다. 이 응답을 활용해 학생 모델을 효율적으로 학습시킬 수 있습니다.

### (1) 교사 모델 로드

이 단계에서는 **교사 모델(Teacher Model)**을 로드합니다.
교사 모델은 고성능 LLM으로, 데이터셋의 기존 답변보다 더 높은 품질의 응답을 생성해줍니다.
이 응답을 이용하여 학생 모델을 간접 학습(=지식 증류) 시킬 수 있습니다.

<br>

Unsloth 라이브러리를 통해 float16 + 4bit 양자화 설정으로 로드하면 일반 GPU 환경에서도 VRAM을 절약하며 실행할 수 있습니다.

<br>

본 실습에서 교사 모델로 사용할 llama-3-Korean-Bllossom-8B는 한국어로 특화된 SFT 모델로, 법률 QA 같은 포멀한 태스크에서 높은 응답 품질을 기대할 수 있습니다.



In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer

# ✅ 사용할 교사 모델 이름 지정
teacher_model_name = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

# ✅ 모델 로드 (float16 + 4bit 양자화로 일반 GPU 호환)
teacher_model, teacher_tokenizer = FastLanguageModel.from_pretrained(
    model_name=teacher_model_name,
    max_seq_length=2048,       # 최대 입력 길이
    dtype=torch.float16,       # float16 사용 (GPU 효율화)
    load_in_4bit=True          # 4bit 양자화로 VRAM 절약
)

# ✅ 토크나이저 설정
teacher_tokenizer.pad_token = teacher_tokenizer.eos_token
teacher_tokenizer.padding_side = 'left'

print(f"교사 모델 '{teacher_model_name}' 로드 완료")

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

교사 모델 'MLP-KTLim/llama-3-Korean-Bllossom-8B' 로드 완료


### (2) 응답 필터링 함수 정의


이 단계는 교사 모델이 생성한 응답 중 품질이 낮은 것들을 걸러내기 위한 유효성 검사 함수를 정의하는 부분입니다.
즉, 다음과 같은 비정상 응답을 제거합니다:
- 너무 짧거나 너무 긴 응답
- 프롬프트 태그([INST], [/INST])가 남아있는 경우
- 같은 문단이 반복되는 경우
- 특수문자가 과도하게 반복된 응답
- 영어 프롬프트("assistant", "please")가 포함된 응답

👉 이런 기준을 통과한 응답만을 증류 학습용 데이터로 사용합니다. 이는 학생 모델의 노이즈 없는 학습을 가능하게 합니다.

In [ ]:
def is_valid_response(response):
    # 응답 길이 제한
    if len(response) < 50 or len(response) > 1000:
        return False
    # 프롬프트 형식이 섞여있으면 제거
    if response.count("[/INST]") > 1 or response.count("[INST]") > 0:
        return False
    # 반복 문단 필터링
    paragraphs = [p.strip() for p in response.split('\n') if p.strip()]
    if len(paragraphs) > 5 and len(set(paragraphs)) / len(paragraphs) < 0.7:
        return False
    # 특수문자 연속 사용 필터링
    special_chars = "!@#$%^&*()_+-=[]{}|;:,./<>?"
    consecutive_special = 0
    for char in response:
        if char in special_chars:
            consecutive_special += 1
            if consecutive_special > 5:
                return False
        else:
            consecutive_special = 0
    # 영어 프롬프트 남아있는 경우
    if response.lower().startswith("assistant") or "please" in response.lower():
        return False
    return True

### (3) 증류 데이터 생성 및 샘플 확인


교사 모델이 생성한 응답을 기반으로 증류 학습에 사용할 데이터를 구성합니다.  
단순히 생성한 응답을 그대로 사용하는 것이 아니라, 응답의 **품질 필터링 조건**을 통과한 경우에만 이를 사용하고, 그렇지 않은 경우에는 원래 데이터셋의 정답(answer)을 사용하도록 설계되어 있습니다.  
이를 통해 데이터 품질을 높이고, 불필요한 노이즈를 줄여 학생 모델이 더 안정적으로 학습할 수 있도록 합니다.

In [ ]:
from tqdm import tqdm

def create_distillation_dataset(teacher_model, teacher_tokenizer, dataset, num_samples=100, batch_size=16):
    teacher_tokenizer.padding_side = 'left'

    questions, original_answers, generated_answers = [], [], []

    # 시스템 프롬프트 - 모델의 응답 일관성을 위한 안내 문구
    system_prompt = "당신은 한국어 법률 전문가입니다. 질문에 대해 정확하고 명료하게 답변해주세요."

    for i in tqdm(range(0, min(num_samples, len(dataset)), batch_size)):
        batch_end = min(i + batch_size, num_samples)
        batch_questions = [dataset[j]["question"] for j in range(i, batch_end)]
        batch_answers = [dataset[j]["answer"] for j in range(i, batch_end)]

        prompts = [f"<s>[INST] {system_prompt}\n\n{q} [/INST]" for q in batch_questions]

        # 토큰화 후 모델 입력으로 변환
        inputs = teacher_tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024
        ).to(teacher_model.device)

        # 교사 모델로부터 응답을 생성하고, 유효성 검사를 통해 정제
        with torch.no_grad():
            outputs = teacher_model.generate(
                **inputs,
                max_new_tokens=512,  # 생성 최대 토큰 수
                temperature=0.2,         # 정확한 응답 유도
                repetition_penalty=1.3,   # 반복 방지
                do_sample=True,
                pad_token_id=teacher_tokenizer.eos_token_id
            )

        for b, output in enumerate(outputs):
            full = teacher_tokenizer.decode(output, skip_special_tokens=True)
            prompt_len = len(prompts[b])
            response = full[prompt_len:].strip()

            # 응답이 유효하면 사용, 그렇지 않으면 원래 정답 사용
            if is_valid_response(response) and len(response) >= len(batch_answers[b]) * 0.5:
                generated = response
            else:
                generated = batch_answers[b]

            questions.append(batch_questions[b])
            original_answers.append(batch_answers[b])
            generated_answers.append(generated)

        torch.cuda.empty_cache()

    print(f"원본 답변 사용 비율: {sum([g==o for g,o in zip(generated_answers, original_answers)]) / len(generated_answers) * 100:.2f}%")

    return {"question": questions, "answer": generated_answers}

앞에서 정의한 `create_distillation_dataset` 함수를 호출하여 교사 모델이 생성한 응답을 바탕으로 증류 데이터셋을 생성합니다.  


전체 500개의 샘플을 추출하여 증류 데이터셋을 만들고, 응답 유효성 검사를 통과한 경우에는 **교사 모델의 응답을 사용**, 그렇지 않으면 **원래 정답(answer)** 을 사용하는 방식입니다.

이후 생성된 증류 데이터셋의 질문-응답 쌍을 출력하여 실제 데이터가 잘 구성되었는지를 확인합니다.


In [ ]:
# 증류 데이터셋 생성
distilled_dataset = create_distillation_dataset(
    teacher_model,
    teacher_tokenizer,
    dataset,
    num_samples=500,      # 학습 시간 고려해 소규모로 시작
    batch_size=16
)

# 샘플 출력
print("\n생성된 샘플 확인:")
for i in range(3):
    print(f"\n샘플 {i+1}:")
    print("질문:", distilled_dataset['question'][i][:100], "...")
    print("답변:", distilled_dataset['answer'][i][:100], "...")

100%|██████████| 32/32 [1:37:31<00:00, 182.86s/it]

원본 답변 사용 비율: 75.60%

생성된 샘플 확인:

샘플 1:
질문: 최저임금제도란 무엇이며, 최저임금액은 어떻게 결정되는지요? ...
답변: 최저임금제도란 국가가 임금액의 최저한도를 정하여 사용자에게 이를 준수하도록 강제하는 제도를 말합니다. 그러므로 최저임금액이 결정·고시되면 사용자는 근로자와 합의하여 최저임금액보다  ...

샘플 2:
질문: 저는 상시 근로자수 13인인 甲회사에서 1983년 10월부터 근무하다가 1개월 전 퇴직하였으나, 甲회사는 퇴직금지급규정이 없다는 이유로 퇴직금을 지급하지 않고 있습니다. 저는 퇴직 ...
답변: 퇴직금제도에 관하여 「근로기준법」제34조에 의하면 「근로자퇴직급여 보장법」이 정하는 바에 따르도록 하고 있으며, 사용자는 근로자퇴직급여보장법에서 정하고 있는 퇴직급여제도 중 하나  ...

샘플 3:
질문: 저는 甲회사에 입사하여 5년째 되던 해 소속부서 업무가 乙회사로 독립되자 甲회사에서 일방적으로 근로자들을 일괄 사직처리하고 퇴직금을 수령하도록 한 후, 그 다음 날짜로 乙회사에 입 ...
답변: 기업의 합병·분할·영업양도 등의 경우 근로자들이 조직변경 전후에 계속하여 근무를 하되, 일단 근로자들이 종전의 기업에서 퇴직하고 그 근무연수에 해당하는 퇴직금을 지급받은 후 새로운 ...


- 생성된 응답 중 **약 75.60%**가 교사 모델의 응답으로 채택되었고, 나머지는 원래 정답을 사용했습니다.

- 이는 교사 모델이 상당히 높은 품질의 응답을 생성했으며, 필터링 조건을 대체로 만족했음을 의미합니다.

- 실제 샘플을 보면 응답이 비교적 자연스럽고 법률 정보를 포함한 내용으로 구성되어 있어 학습 데이터로 사용하기에 적절합니다.

In [ ]:
# GPU 메모리를 확보하기 위해 교사 모델 객체를 삭제하고 캐시 메모리를 정리

del teacher_model
gc.collect()
torch.cuda.empty_cache()

## 3) 학생 모델 로드 및 학습 준비

### (1) 학생 모델 로드 및 LoRA 설정


학생 모델은 증류 대상이 되는 모델로, Unsloth를 활용하여 4bit 양자화된 상태로 로드하며, LoRA를 적용하여 효율적인 경량 학습이 가능하도록 설정합니다.  
교사 모델보다 작은 모델을 사용하므로, VRAM 자원이 제한된 환경에서도 안정적으로 학습할 수 있습니다.

In [ ]:
from datasets import Dataset
from transformers import TrainingArguments
from trl import SFTTrainer

# 사용할 학생 모델 이름 지정 (경량 LLM)
student_model_name = "meta-llama/Llama-3.2-1B"

# 학생 모델 및 토크나이저 로드
student_model, student_tokenizer = FastLanguageModel.from_pretrained(
    model_name=student_model_name,
    max_seq_length=2048,
    dtype=torch.float16,  # 일반 GPU 사용을 위한 float16
    load_in_4bit=True     # 4bit 양자화로 VRAM 절약
)

# 토크나이저 설정
student_tokenizer.pad_token = student_tokenizer.eos_token

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

### (2) 학습 전 추론 테스트


학생 모델을 불러온 직후, 학습(Lora fine-tuning)되지 않은 기본 상태에서의 응답 품질을 확인합니다.  
이 결과는 추후 증류 학습 후 얼마나 개선되었는지를 비교하기 위한 기준선(Baseline)이 됩니다.

In [ ]:
# 질문 샘플 정의
sample_question = "최저임금제도란 무엇이며, 최저임금액은 어떻게 결정되는지요?"

# 프롬프트 구성 (Alpaca 스타일)
prompt = f"""<|system|>
당신은 한국어 법률 전문가입니다. 질문에 대해 정확하고 명료하게 답변해주세요.
</|system|>

<|user|>
{sample_question}
</|user|>

<|assistant|>
"""

# 토크나이저로 인코딩
inputs = student_tokenizer(prompt, return_tensors="pt").to(student_model.device)

# 추론 실행
with torch.no_grad():
    output = student_model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.2,
        do_sample=True,
        pad_token_id=student_tokenizer.eos_token_id
    )

# 출력 디코딩 및 정리
response = student_tokenizer.decode(output[0], skip_special_tokens=True)
generated_answer = response.split("<|assistant|>")[-1].strip()

# 결과 출력
print("질문:", sample_question)
print("\n[학습 전 기본 학생 모델 응답]:")
print(generated_answer)

질문: 최저임금제도란 무엇이며, 최저임금액은 어떻게 결정되는지요?

[학습 전 기본 학생 모델 응답]:
최고임금을 정하는 데는 여러 가지 방법이 있습니다. 대한민국에서 최고임금을 결정하기 위해 하는 방법중 하나는 2013년 제정된 기본근로인건평균급여법의 규정을 따르도록 하였습니다.

**《대기근소지업계규격조례》와 《대기생산단위산업협회》를 통한 근소지업감축 계획**

각 협회의 회원들은 각 연간 감축계획안을 제출하여 그에 대한 승인을 받아야 합니다. 이승인은 각 협회의 자원봉사단체위원회에서 지체없이 검토하며, 최종적으로 심사를 거쳐서 각 협회의 회장에게 넘겨 받게 됩니다. 또한 각 협회에서는 그 세월마다 감축가능성 점검보고서를 작성할 것이라고 하고, 이를 수시로 보수심사(현 고용재직기간) 등에 기초 삼아서 주어진 일정 기간동안 체결한다고 약속됩니다.
또한 여기에는 국가기관이나 지방자치구의 특별청구권과 관련해서는 조약을 맺으면 안된다며 특히 상납비가 없는 경우나 특허적 또는 무역특화 산업으로 기술되어 있고 부동산세 및 경영정보보호등에 관한 특혜 등에 불참하라는 것도 있겠습니다.

국내외에서 기업들이 참여하면서 사업들을 위한 다양한 조합이 만들어진 것이죠. 그리고 현재까지는 많은 국내기업들 중에서도 2008년에 새누리당 정부가 실시해 온 노조노선사업 시설공작업추진조달방편(새누리당정부의 노조노선사업)에 의거해 공공부문 전력사업분야의 노조투입을 통해 생산능력을 향상시키면서 사업유지를 확대하여 새로운 역전기를 시작했습니다. 최근에도 같은 취지를 가지고 있는 국제무역거래처협회 등 세계적인 주요무역항목협회들의 행렬 속에서 일본 회사들이 더욱 강행되고 있는 바와 같이 우리나라 대형무역업체들도 국제무역업에 적극 투자를 위해 유엔총연맹이 발족하는 한미교섭협회 등을 모색하고 있습니다.
그렇기에 일일경쟁이라는 문제점이 생략되겠지만 결론부터 이야기하자면 저희 한국기업들께서는 해외무


결과를 보면 기본적으로 한국어로 답변을 하지만 아쉬운 부분도 많이 보여주고 있습니다.

- 비현실적 응답
- 맥락 불일치
- 혼란스러운 문맥
- 지식 정확도 낮음

이제 모델에 LoRA를 적용하여 경량 학습 설정을 구성합니다.

In [ ]:
# LoRA 파라미터 설정 및 모델 구성
student_model = FastLanguageModel.get_peft_model(
    student_model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],  # 학습할 계층
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.3.19 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


### (2) 학습 데이터 구성 (ChatML 포맷 기반 텍스트 구성)


이 실습에서는 ChatML 스타일의 대화형 프롬프트 포맷(<|system|>, <|user|>, <|assistant|>)을 기반으로 학습 텍스트를 구성합니다.

ChatML 포맷은 Meta의 LLaMA 3 계열 모델에서 사용하는 구조이며, 사용자/시스템 발화 구분을 명확히 할 수 있어 효과적인 fine-tuning이 가능합니다.

In [ ]:
# HuggingFace Datasets 형식으로 변환
hf_dataset = Dataset.from_dict(distilled_dataset)

# 학습/검증 데이터 분리
split = hf_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split["train"]
eval_dataset = split["test"]

# 프롬프트 템플릿 정의
TEMPLATE = """<|system|>
당신은 한국어 법률 전문가입니다. 질문에 대해 정확하고 명료하게 답변해주세요.
</|system|>

<|user|>
{question}
</|user|>

<|assistant|>
{answer}"""

# 텍스트 포맷 변환 함수 정의
def format_dataset(example):
    return {"text": TEMPLATE.format(question=example["question"], answer=example["answer"])}

# 데이터셋 포맷 변환
train_dataset = train_dataset.map(format_dataset)
eval_dataset = eval_dataset.map(format_dataset)

# 변환 샘플 확인
print("\nAlpaca 포맷 샘플:")
print(train_dataset[0]["text"])


Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]


Alpaca 포맷 샘플:
<|system|>
당신은 한국어 법률 전문가입니다. 질문에 대해 정확하고 명료하게 답변해주세요.
</|system|>

<|user|>
노동조합의 대표자인 甲은 행정관청으로부터 노동조합의 결산결과와 운영상황을 보고하라는 요구를 받았습니다. 이에 甲은 관련 민원이 제기되거나 조직분규가 발생하는 등 행정관청이 위 자료를 요구할 업무상 필요성이 인정되지 않는다며 이를 거부하고 있습니다. 甲의 자료제출 거부는 정당한 것인가요?
</|user|>

<|assistant|>
노동조합 및 노동관계조정법 제27조는 노동조합은 행정관청이 요구하는 경우에는 결산결과와 운영상황을 보고하여야 한다고 규정하고 있습니다. 행정관청이 이와 같은 자료제출을 요구할 업무상 필요성이 인정되는 경우로는 노동조합 운영과 관련하여 조합원 등 이해관계인의 진정·고발·청원 등 민원이 제기된 경우, 노동조합의 조직분규가 있어 이를 조정할 필요가 있는 경우, 회의소집권자 지명요구, 결의·처분 시정요구, 노조가입·탈퇴와 관련하여 이의제기 등과 관련하여 사실관계의 확인이 필요한 경우 등이 있습니다.그런데 위와 같이 노동조합에 대하여 업무지도의 필요가 있는 경우에 해당되지 않은 경우에 관하여도 대법원은 “설사 노동조합의 회계 경리상태나 기타 운영에 대하여 지도할 필요가 있는 경우에 해당되지 않는다고 하더라도 행정관청이 그와 같이 판단하여 조사하기로 한 이상 노동조합은 이에 응할 의무가 있다고 할 것이며, 행정관청이 피조사자인 노동조합에 대하여 조사이유나 근거에 대하여 설명하지 아니하였다고 하여 노동조합이 이에 응하지 아니할 정당한 이유가 있다고 할 수는 없다.”고 판단하였습니다(대법원 1993. 4. 23. 선고 92도2818 판결).


## 4) 학생 모델 학습 및 저장

### (1) 학습 설정 및 진행


학생 모델에 대해 LoRA 기반 경량화 파인튜닝 설정을 구성하고, Unsloth의 SFTTrainer를 활용하여 학습을 실행합니다.

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# LoRA 파인튜닝 설정
student_model = FastLanguageModel.get_peft_model(
    student_model,
    r=16,  # LoRA 랭크
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],  # 학습할 계층
    lora_alpha=16,   # LoRA scaling factor
    lora_dropout=0,  # 드롭아웃 없음
    bias="none",
    use_gradient_checkpointing="unsloth",  # VRAM 절약
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir="./student_model_distilled",  # 모델 저장 경로
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,  # 총 batch size = 4 x 4 = 16
    learning_rate=2e-4,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    warmup_steps=50,
    max_steps=200,  # 실습을 위한 짧은 학습
    fp16=True,
    bf16=False,
    optim="adamw_8bit",  # 8bit 옵티마이저
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=42,
    report_to="none",  # 로그를 콘솔에만 출력
)

# SFTTrainer 설정 및 학습 시작
trainer = SFTTrainer(
    model=student_model,
    tokenizer=student_tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    args=training_args,
    packing=False,  # 여러 문장을 하나로 합치지 않음
)

print("✅ 학생 모델 학습 시작...")
trainer.train()


Unsloth: Already have LoRA adapters! We shall skip this step.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/450 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

학생 모델 학습 시작...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 450 | Num Epochs = 8 | Total steps = 200
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
50,1.835200,1.889193
100,1.462600,1.736847
150,1.272500,1.703883
200,1.288300,1.700554


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=200, training_loss=1.5976203966140747, metrics={'train_runtime': 1453.4494, 'train_samples_per_second': 2.202, 'train_steps_per_second': 0.138, 'total_flos': 1.6855737849298944e+16, 'train_loss': 1.5976203966140747})

모델이 점진적으로 교사 응답을 학습하고 있는것을 확인할 수 있습니다. 다만 본 실습은 간단한 테스트이므로 짧은 학습만 진행하고 다음 단계로 넘어가겠습니다.

(학습량을 늘리거나 증류 데이터의 품질을 높일 경우 더 나은 성능을 기대할 수 있습니다.)

### (2) 모델 저장
학습이 완료된 후, 모델과 토크나이저를 저장합니다.

In [ ]:
# 모델 저장
print("모델 저장 중...")
student_model.save_pretrained("./student_model_distilled/final")
student_tokenizer.save_pretrained("./student_model_distilled/final")
print("학생 모델 저장 완료!")

모델 저장 중...
학생 모델 저장 완료!


## 5) 증류 결과 확인 및 비교

### (1) 증류 전후 응답 비교

학생 모델이 학습을 통해 얼마나 개선되었는지 확인하기 위해, 동일한 질문에 대해 증류 전 (원본), 교사 모델, 학생 모델의 응답을 비교해보는 것이 좋습니다.
아래 코드는 한 개의 질문을 입력으로 세 모델의 응답을 출력합니다. (교사 모델은 이미 삭제했기 때문에 생략 가능)

우선 저장된 학생 모델을 다시 로드합니다:

In [ ]:
from transformers import AutoTokenizer, TextStreamer
from unsloth import FastLanguageModel

# 저장된 LoRA 학습된 학생 모델 불러오기
student_model, student_tokenizer = FastLanguageModel.from_pretrained(
    model_name="./student_model_distilled/final",
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True
)

student_tokenizer.pad_token = student_tokenizer.eos_token
student_tokenizer.padding_side = "left"

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
unsloth/llama-3.2-1b-unsloth-bnb-4bit does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


질문을 입력하고 학생 모델의 응답을 확인합니다:

In [ ]:
# 비교할 질문 샘플
sample_question = "최저임금제도란 무엇이며, 최저임금액은 어떻게 결정되는지요?"

# 프롬프트 구성
prompt = f"""<|system|>
당신은 한국어 법률 전문가입니다. 질문에 대해 정확하고 명료하게 답변해주세요.
</|system|>

<|user|>
{sample_question}
</|user|>

<|assistant|>
"""

inputs = student_tokenizer(prompt, return_tensors="pt").to(student_model.device)

# 모델 응답 생성
with torch.no_grad():
    output = student_model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.2,
        do_sample=True,
        pad_token_id=student_tokenizer.eos_token_id
    )

response = student_tokenizer.decode(output[0], skip_special_tokens=True)
# 응답 부분만 추출
generated_answer = response.split("<|assistant|>")[-1].strip()

print("질문:", sample_question)
print("\n[학생 모델 응답]:")
print(generated_answer)

질문: 최저임금제도란 무엇이며, 최저임금액은 어떻게 결정되는지요?

[학생 모델 응답]:
안녕하세요! 저희는 ‘법적으로’ 말하지만, 저희의 의견대로하겠습니다(이른바 “정치적” 표현). 

1) **최저임금**:
    - 일반적인 경우에는 임금보다 낮아야 할 수준을 갖추게 됩니다.
2) **재량권**:
    - 특수한 사유에서 이를 조정할 수도 있지만..
3) **협상과정**：
    - 협상을 통해 합리적인 기준점을 설정합니다.

4) **구두명령**：
    - 구두명을 해줘야 합니다.


5) **일반규칙**： 
    - 일반 규정이 있습니다.



6) **예외처리가**：

    - 예외 처리 가능성이 있어.


7) **종합평산**：
    - 종합평산되어요!

8) **다양성능력**：
    - 다양성을 위해 노력을 하며...


9) **복잡화면역작용**：
    - 복소화를 위한 작업들을 필요로 하는 것 같습니다...



10) **반대방향접근**：
    - 반대의 방향 접근하는 것을 고려해야 합니다...

11) **통일된결과**：
     - 통일에 도달하여 완벽히 이루어진다면...
12) **미지배되지 않는경위**：
      - 미지가 발생하지 않을 때까지 기다립니다....
13) **무효부여없는경우**：
       - 무효 부여 없는 경위를 가지 않습니다.....

14) **완전히필립될경기**：
         - 완전 필립 될 기분 없습니다....

15) **마지막단절**：
          - 마지막 단절 까지는 남겨진 부분만큼됩니다......

16) **사실내장**：
           - 실체 내장은 없으며 없어짐니다....


17) **종국적으로**：
                - 종국적으로 끝난다는 것은 확실합니다.....

18) **충족되었습니다**：
                 - 충족 되었습니다 이는 모든 것이 만족되고 완벽하였습니다......
19) **자연속성 유지**：
             

일부 항목은 문장 구조가 개선되었으나, 사실적 정확도나 응답의 명료성은 여전히 부족한 수준입니다.

이는 학습량, LoRA 설정, 프롬프트 구조 등이 모두 실험적으로 최소화되어 있기 때문입니다.

### (2) 간단한 추론 테스트

여러 질문을 리스트로 넣어 테스트할 수도 있습니다. 아래는 예시입니다:

In [ ]:
test_questions = [
    "회사에서 퇴직금을 지급하지 않으면 어떻게 해야 하나요?",
    "근로계약서가 없을 경우 임금청구가 가능한가요?",
    "정규직과 비정규직의 차이는 무엇인가요?"
]

for i, q in enumerate(test_questions):
    print(f"\n질문 {i+1}: {q}")

    prompt = f"""<|system|>
당신은 한국어 법률 전문가입니다. 질문에 대해 정확하고 명료하게 답변해주세요.
</|system|>

<|user|>
{q}
</|user|>

<|assistant|>
"""

    inputs = student_tokenizer(prompt, return_tensors="pt").to(student_model.device)
    with torch.no_grad():
        output = student_model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            top_p=0.95,
            repetition_penalty=1.2,
            do_sample=True,
            pad_token_id=student_tokenizer.eos_token_id
        )
    response = student_tokenizer.decode(output[0], skip_special_tokens=True)
    generated_answer = response.split("<|assistant|>")[-1].strip()
    print("[응답]:", generated_answer)


질문 1: 회사에서 퇴직금을 지급하지 않으면 어떻게 해야 하나요?
[응답]: 퇴직금청구권이 인정되는 경우에는, 근로자에게 임의의 시기 또는 방법으로 그 소유물의 매각·전속승계 기타 원래 사용자의 귀책사실 없이 그의 손길로 발생하는 이익과 손실 중간점을 전제하여 청구할 수 있습니다(근로기준법 제42조).그러므로 이러한 관행상, 회사에서는 사후적 퇴직금 지급을 하지 않는다고는 하며, 이는 근거없이 허용될 수 없으므로 사실대로 고려해야 할 것입니다.                   

1) **임시**: 일시적인 조건 아래서만 퇴직 금지를 정하기 위해서라도, 이를 통해 위험한 경쟁사업이나 사업 등을 방지시키고, 새로운 경영관리를 실천하려면 필요한데다, 회사가 계속적으로 성장하거나 확대나 개편되면서도야 할 필요 등에도 기인됩니다.(2)
3)를 모두 만족하면 된다.

4)에 따라 퇴직금청구권이 인정된다.


5)의 기준:

6)이란:
7)을 요구한다.




8)은 다음과 같다:


9)는


10)는
11）は

질문 2: 근로계약서가 없을 경우 임금청구가 가능한가요?
[응답]: 노동부장관이 정한 근로기준법의 적용대상인으로 하여야 하는 근로자의 한 사람이라면, 그 이외의 근로자들은 근로관계를 종식시키고 통상의 경쟁적 관계에서 벗어나기 위하여는 특별히 필요한 조건없이 당사자는 사용자와 단체협약 등 사이에 노무교환형태로 계속된 근로조건 및 임금채권의 유지나 채권변제권과 동시에 새로운 근로조건이나 임금채권을 형성할 수 있는 합의를 하고 있어야 합니다(대법원 1994.12.23. 선고 93다카1203 판결). 따라서 기본적으로 임금이 청구하지 못한다 할 것입니다.                    다만, 노동부령 (2012.10.30.) 제1097호에서는 ‘사업주의 지배·관리 등을 위한 기간 중임금을 목적으로 계약하거나 고용되어 있다면 이를 포함시켜 원칙대로 계산하는 것이 타당하다.’라고 하고 있으며, 같은 규정 본문에서도 “근로자가 사업주에게 일정기간에 걸쳐 임금 기타 보

역시 부족한 결과를 보여줍니다.

---

이 실습은 구조와 흐름을 이해하기 위한 것이므로,
실제 성능 개선을 원한다면 아래와 같은 방향으로 개선해보세요:

- 학습 데이터 수 증가
- 학습량 자체의 증가
- 모델 변경
- 응답이 과도하게 길어지지 않도록 max_new_tokens, repetition_penalty 조절
- 다양한 질문을 입력해 모델의 강점/약점을 파악해보기